In [1]:
#RUN ONLY ONCE
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import ctypes  # An included library with Python install.
import datetime
from dateutil.relativedelta import *

currentDT = datetime.datetime.now()

def Mbox(title, text, style):
    return ctypes.windll.user32.MessageBoxW(0, text, title, style)

#Instructions: Fill in your details in the config, run the script, solve the captcha, and wait.

#CONFIGURATION
CHROME_DRIVER_DOWNLOADED_PATH = 'C:/Users/user/Desktop/selenium/chromedriver.exe'
GLOBAL_SLOWDOWN = 1 #Do not edit.
CAPTCHA_WAIT = 100 #Means you have 100 seconds to do the captcha
REFRESH_WAIT = random.randint(269, 342) #If you refresh too quickly the site will block you for a few hours.
OFFICE_NAME = "OAKLAND COLISEUM"
# OFFICE_NAME = "OAKLAND CLAREMONT"
# OFFICE_NAME = "HAYWARD"
# OFFICE_NAME = "EL CERRITO"
# OFFICE_NAME = "OAKLAND CLAREMONT"
FIRST_NAME = "BOB"
LAST_NAME = "JONES"
DL_NUMBER = "Y1234567"
BIRTH_MONTH = "1"
BIRTH_DAY = "1"
BIRTH_YEAR = "1996"
TEL_AREA = "510"
TEL_PREFIX = "123"
TEL_SUFFIX = "4567"
GMAIL = "youremail@gmail.com"
GMAIL_PASS = "yourpass"
EMAIL_TARGET_ADDRESSES = ['tosendto@gmail.com']

#Enter 1 if you're running on windows for the popup, otherwise enter 0
WINDOWS = 0


#Used to prompt the site. Enter a valid weekday after today.
#Update: Date is now automatically input
#NEXT_WEEKDAY_DAY_MONTH_YEAR = 'THU, 19 Sep 2019'

def format_date_input():
    if currentDT.strftime("%a") == 'Fri':
        nextDT = currentDT+datetime.timedelta(days=3)
    else:
        nextDT = currentDT+datetime.timedelta(days=1)
    date_input = nextDT.strftime("%a").upper() + ', ' + nextDT.strftime("%d") + ' ' + nextDT.strftime("%b") + ' ' + nextDT.strftime("%Y")
    print ('ping check datetime = ', end = '')
    print (date_input)
    
    thismonth = nextDT.strftime("%b")
    next_month_DT = nextDT+relativedelta(months=+1)
    nextmonth = next_month_DT.strftime("%b")
    
    return (date_input, thismonth, nextmonth)

FORMATTED_NEXT_WEEKDAY_DAY_MONTH_YEAR, thismonth, nextmonth = format_date_input()
NEXT_WEEKDAY_DAY_MONTH_YEAR = FORMATTED_NEXT_WEEKDAY_DAY_MONTH_YEAR

#Optional:
#Months available to trigger alert, enter exactly 2 months in same format. 
#Leave it as it is to find appointments in this month or next month.
appointment_date_alert_filter = [thismonth, nextmonth]
print('appointment date alert filter = ', end = '')
print(appointment_date_alert_filter)
timecheck = '8:00 AM'

#Remember to enable less secure login via gmail settings to allow this to work.
#Copy file and run different OFFICE_NAME(s) on different tabs to run them simultaneously
##############################################################################################################################
driver = webdriver.Chrome(executable_path = CHROME_DRIVER_DOWNLOADED_PATH)
driver.get('https://www.dmv.ca.gov/wasapp/foa/driveTest.do')
assert "Make Appointment" in driver.title

#SET DMV OFFICE
element = driver.find_element_by_name('officeId')
element.send_keys(OFFICE_NAME)
time.sleep(GLOBAL_SLOWDOWN)

#SET TEST TYPE AS CAR
radioElement = driver.find_element_by_id("DT")
radioElement.click()
time.sleep(GLOBAL_SLOWDOWN)

#SET FIRST NAME
element = driver.find_element_by_name('firstName')
element.send_keys(FIRST_NAME)
time.sleep(GLOBAL_SLOWDOWN)

#SET LAST NAME
element = driver.find_element_by_name('lastName')
element.send_keys(LAST_NAME)
time.sleep(GLOBAL_SLOWDOWN)

#SET PERMIT LICENSE NUMBER
element = driver.find_element_by_name('dlNumber')
element.send_keys(DL_NUMBER)
time.sleep(GLOBAL_SLOWDOWN)

#SET BIRTH DATE MM
element = driver.find_element_by_name('birthMonth')
element.send_keys(BIRTH_MONTH)
time.sleep(GLOBAL_SLOWDOWN)
#SET BIRTH DATE DD
element = driver.find_element_by_name('birthDay')
element.send_keys(BIRTH_DAY)
#SET BIRTH DATE YYYY
time.sleep(GLOBAL_SLOWDOWN)
element = driver.find_element_by_name('birthYear')
element.send_keys(BIRTH_YEAR)

time.sleep(GLOBAL_SLOWDOWN)
#SET PHONE AREA CODE
element = driver.find_element_by_name('telArea')
element.send_keys(TEL_AREA)
time.sleep(GLOBAL_SLOWDOWN)
#SET PHONE NEXT 3 DIGITS
element = driver.find_element_by_name('telPrefix')
element.send_keys(TEL_PREFIX)
time.sleep(GLOBAL_SLOWDOWN)
#SET PHONE LAST 4 DIGITS
element = driver.find_element_by_name('telSuffix')
element.send_keys(TEL_SUFFIX)

time.sleep(GLOBAL_SLOWDOWN)

#NEXT PAGE
driver.find_element_by_xpath("//input[@type='submit' and @value='Continue']").click()

time.sleep(CAPTCHA_WAIT)
count = 0

while (1):
    flag = 0
    print('try', count)
    try:
        element = driver.find_element_by_css_selector('input.ng_cal_input_field')
        element.send_keys(NEXT_WEEKDAY_DAY_MONTH_YEAR)
        element = driver.find_element_by_name('requestedTime')
        element.send_keys(timecheck)
        time.sleep(GLOBAL_SLOWDOWN)
        driver.find_element_by_xpath("//input[@type='submit' and @value='Check for Availability']").click()
        el=driver.find_element_by_css_selector("table > tbody td[data-title='Appointment']")  
        print(el.text)
    except KeyboardInterrupt:
        raise
    except:
        element.send_keys(Keys.F5)
        print('Unable to find date')
        time.sleep(REFRESH_WAIT)
        continue
    time.sleep(REFRESH_WAIT)
    count+=1
    for month in appointment_date_alert_filter:
        if month in el.text:
            flag = 1
            break
    time.sleep(GLOBAL_SLOWDOWN)
    
    avail_date = el.text[99:]
    print(avail_date)


import smtplib

if flag == 1:
    print('sending email')
    gmail_user = GMAIL
    gmail_password = GMAIL_PASS

    sent_from = GMAIL
    to = EMAIL_TARGET_ADDRESSES
    subject = 'DMV Appointment Available'
    body = 'Get the fk up and sign up for the driving test now \n' + 'APPOINTMENT FOUND ON ' + avail_date

    email_text = """\
    From: %s \n\n
    To: %s \n\n
    Subject: %s \n\n

    %s
    """ % (sent_from, ", ".join(to), subject, body)
    
# TODO: #Automated booking
#     try:
#         print('do nothing')
#         avail_date = el.text[99:]
#         print('the available date is ' + str(avail_date))
        
        
        
#     except:
#         print('unable to book appointment automatically')
#         pass()
    
    
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        time.sleep(2)
        server.sendmail(sent_from, to, email_text)
        server.close()
        if windows:
            Mbox('BOOK DMV APPOINTMENT', 'APPOINTMENT FOUND ' + el.text, 1)
        print ('Email sent!')
    except:
        print ('Something went wrong...')

driver.close()


ping check datetime = MON, 14 Oct 2019
appointment date alert filter = ['Oct', 'Nov']
try 0
Sorry, no appointment is available for the selected time. Another appointment time is available on:
Jan 7, 2020 at 12:10 PM
